# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: Other formats

Michael Reefe

This example notebook provides a quick tutorial on how to run LOKI using non-MIRI data.

Just like in the MIRI case, we first need to import the LOKI code.  Everything noted in the MIRI notebook is still relevant here, including the explanations on how to enable multiprocessing with the `Distributed` module.

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()

using Loki

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Now we want to load in our data. For this example, we'll be using an SDSS optical spectrum of a red galaxy (to showcase the utility of the stellar continuum fitting functionality), which is located in the same folder as this notebook. Since this is not a JWST MIRI formatted data product, there is no built-in functionality for LOKI to read its contents and convert it into a data format that it can work with.  This is also notably not an IFU data cube, but just a single spectrum.  This is fine, as we can effectively treat it as an IFU cube with spatial dimensions of 1x1.  We'll start by reading in the FITS file with Julia's `FITSIO` module and storing relevant information in arrays.

In [2]:
using FITSIO                    # for reading/writing FITS files
using Unitful, UnitfulAstro     # to add units to the data

In [3]:
# Read in the FITS file and header
hdu = FITS("spec-0266-51602-0001.fits")
hdr = read_header(hdu[1])

# Wavelength vector (angstrom)
λ = 10 .^ read(hdu["COADD"], "loglam") .* u"angstrom"

# Flux cube (erg/s/cm^2/ang)
F = read(hdu["COADD"], "flux") .* 1e-17 .* u"erg/s/cm^2/angstrom"
# Reshape to a 1 x 1 x (n_wavelength) cube
F = reshape(F, (1,1,length(F))) 

# Error cube (erg/s/cm^2/ang)
eF = sqrt.(1 ./ read(hdu["COADD"], "ivar")) .* 1e-17 .* u"erg/s/cm^2/angstrom"
# Reshape to a 1 x 1 x (n_wavelength) cube
eF = reshape(eF, (1,1,length(eF)))

# Bad pixel mask 
mask = read(hdu["COADD"], "and_mask") .!= 0
mask = reshape(mask, (1,1,length(mask)))

# Auxiliary information
z = read(hdu["SPECOBJ"], "z")[1]
ra = hdr["RA"]
dec = hdr["DEC"]

# Switches
rest_frame = false    # the input spectrum is not in the rest frame
masked = false        # the bad pixel mask has not been applied (i.e. bad pixels are still in the spectrum)
vacuum_wave = true    # the wavelengths provided are in vacuum wavelengths (SDSS spectra already come in vacuum wavelengths)
dereddened = false    # the spectrum has not been corrected for Milky Way dust absorption along the line of sight
log_binned = true     # the input wavelength vector is logarithmically spaced
sky_aligned = true    # this parameter only makes sense for a full IFU cube -- if true, the input cube has x/y axes aligned with
                      # the RA/Dec axes on the sky, otherwise, it may have any arbitrary orientation

# Since LOKI works in intensities rather than fluxes, we need the solid angle covered
# by each pixel in the input data cube.  Since we only have a single spectrum here,
# we take the solid angle as that which is covered by the SDSS fibers, which have a
# diameter of 3 arcseconds:
Ω = uconvert(u"sr", (π * (3/2)^2)*u"arcsecond^2")

# Then we convert the fluxes into intensities
I = F ./ Ω
σ = eF ./ Ω

# We also need some measurements of the *spectral* and *spatial* resolutions of the input cube.

# Again, in our case since there is only 1 "pixel", we don't really have a concept of spatial resolution. So we'll
# just assume here that the spatial resolution is equal to the size of the aperture:
psf_fwhm = uconvert(u"arcsecond", sqrt(Ω))

# For the spectral resolution, we can use the "wdisp" information stored in the SDSS data:
wdisp = read(hdu["COADD"], "wdisp") # dispersion of each pixel in pixel units
dλ = @. (λ[2]/λ[1] - 1.) * λ        # size of each pixel in wavelength units
fwhm_res = 2.355 .* wdisp .* dλ     # FWHM resolution of each pixel in wavelength units
R = λ ./ fwhm_res                   # spectral resolution (unitless)

# Close the FITS file
close(hdu)

In [4]:
# Create a LOKI Cube object
cube = from_data(Ω, z, λ, I; α=ra, δ=dec, psf_fwhm=psf_fwhm, R=R, wcs=nothing, psf_model=nothing,
                 rest_frame=rest_frame, masked=masked, vacuum_wave=vacuum_wave, dereddened=dereddened, sky_aligned=sky_aligned, log_binned=log_binned)
# Create an Observation object
obs = from_cubes("spec-0266-51602-0001", z, [cube], [0], inst="SDSS")

┌ Warning: The input wavelength vector is non-linear. The wavelength axis of the auto-generated WCS will not accurately reflect this fact. This will not matter for the rest of this code, as it will not be used, but keep this in mind if using FITS files generated as outputs.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/cubedata.jl:485
┌ Warning: The DataCube format is not JWST! Please manually input the channel edges (if any), otherwise it will be assumed that there is only one channel
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/cubedata.jl:562


Observation(Dict{Any, DataCube}(0 => DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[3806.272705078125 Å, 3807.150390625 Å, 3808.026123046875 Å, 3808.904296875 Å, 3809.780517578125 Å, 3810.658935546875 Å, 3811.53564453125 Å, 3812.41259765625 Å, 3813.291748046875 Å, 3814.1689453125 Å  …  9183.3271484375 Å, 9185.439453125 Å, 9187.5576171875 Å, 9189.671875 Å, 9191.78515625 Å, 9193.9052734375 Å, 9196.0205078125 Å, 9198.140625 Å, 9200.2568359375 Å, 9202.37890625 Å], Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}[3.309362892629094e-6 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; 3.7681091858022077e-6 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; 3.58519418212832e-6 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; … ;;; 5.5917133866308534e-6 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; 5.748525031076515e-6 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; 5.5127

In [5]:
channel = 0  # (this is just used as a dummy value when the instrument doesn't have distinct spectral channels)
nm = "spec-0266-51602-0001"
run_name = "$(nm)_example"

"spec-0266-51602-0001_example"

Before fitting, we want to do some pre-processing on the data, just like in the MIRI example.  Exactly which pre-processing steps are performed are largely controlled by the boolean parameters listed above when creating the Cube object (i.e. `rest_frame`, `masked`, `vacuum_wave`, `dereddened`, `log_binned`, and `sky_aligned`). The corrections are actually carried out by calling the `correct!` function.  This is also where we could combine data from multiple channels into a single cube, if desired, using the `combine_channels!` function, but in this quick example we only have one sub-channel.

In [6]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)
    
    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end

[ Info: Using SFD98 dust map at (α=145.89219°, δ=0.059372°): E(B-V)=0.09877200600227425
[ Info: Interpolating NaNs in cube with channel Generic Channel, band Generic Band:
[ Info: Calculating statistical errors for each spaxel...
[ Info: Writing FITS file from Observation object


Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data, same as with the MIRI example, but without an aperture argument since there is only one spaxel anyways.

In [7]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.

cube_fitter = CubeFitter(
    obs.channels[channel],
    obs.z, 
    run_name; 
    parallel=false, 
    plot_spaxels=:both, 
    plot_maps=false,  # <= this disables plotting 2D parameter maps, which we do here because it's a 1x1 cube and maps wouldn't really be informative
    save_fits=true,
    extinction_curve="calz", 
    linemask_width=500.0u"km/s"
)

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter)

[ Info: Preparing output directories
┌ Warning: The maximum wavelength in the input spectrum is < 2 μm! Thermal dust emission will be disabled.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/create_params.jl:330
┌ Warning: There are no PAH features within the input wavelength range of the spectrum. The use_pah_templates option will be disabled.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/create_params.jl:506
[ Info: Generating 400 simple stellar population templates with FSPS with ages ∈ (0.0010000000000000002 Gyr, 13.7 Gyr), log(Z/Zsun) ∈ (-2.3, 0.4)


Progress: 100%|███████████████████████████| Time: 0:18:03 ( 2.71  s/it)


┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR spec-0266-51602-0001_example ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     1
│ Threads per process:  1
└ ------------------------
[ Info: ===> Preparing output data structures... <===
[ Info: Integrating spectrum across the whole cube...
[ Info: ===> Beginning integrated spectrum fitting... <===
[ Info: ===> Generating parameter maps and model cubes... <===
[ Info: ===> Writing FITS outputs... <===
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! ##################################
└ #############################################################################


(CubeFitter{Float32, Int64, Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[3726.9797166572175 Å, 3727.83911809381 Å, 3728.6966070932694 Å, 3729.556486639145 Å, 3730.414453747888 Å, 3731.274572348405 Å, 3732.1330175664307 Å, 3732.9917018390984 Å, 3733.8525376035404 Å, 3734.711460930849 Å  …  8992.018351178973 Å, 8994.086651938193 Å, 8996.160690008812 Å, 8998.230903205165 Å, 9000.300160182951 Å, 9002.376110690704 Å, 9004.447280105624 Å, 9006.523230613375 Å, 9008.59535624686 Å, 9010.673219191745 Å], Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹)

And the results can be found in the "output_[run_name]" directory!
Here is our fit of the SDSS spectrum of spec-0266-51602-0001:

![results_1D](./spec-0266-51602-0001.jpg)

The orange line shows the final model.  The decomposed components of the model consist of:
- Stellar continuum, in pink
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

You will also notice that emission lines get nice labels at the top of the plot.

The stellar population decomposition is also shown across a grid of age and metallicity here:

![results_ssp](./spec-0266-51602-0001.stellar_grid.jpg)

The upper panel shows the weights in luminosity fractions, while the lower panel shows them in mass fractions. As expected for a red galaxy, we have a stellar population of mostly older, metal-poor stars.  The total stellar mass calculated by this model is also shown in the title.